In [9]:
import ifcopenshell, ifcopenshell.geom
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Display.SimpleGui import init_display
import numpy as np
import toposort
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)



In [10]:
def gpTrsf2T(gpT):
    T=np.zeros((4,4))
    for ii in range(3): #gpTrsf defines are 3x4 matrix
        for jj in range(4):
            T[ii,jj] = gpT.Value(ii+1,jj+1)
    T[3,3] = 1 #add the 1 for the [4th,4th place]
    return T

def ifcshow(ifc):
    occ_display, start_display, add_menu, add_function_to_menu = init_display()
    cam = occ_display.View.Camera()
    products = ifc.by_type("IfcProduct")
    for product in products:
        if product.is_a("IfcOpeningElement"): continue
        if product.Representation:
            shape = ifcopenshell.geom.create_shape(settings, product)
            r, g, b, a = shape.styles[0] # the shape color
            color = Quantity_Color(abs(r), abs(g), abs(b), Quantity_TOC_RGB)
            display_shape = occ_display.DisplayShape(shape.geometry,color=color, transparency=abs(1 -a),update=True)
    start_display() #must be used or kernel crashes

def OptimizeIFC(f):
    g = ifcopenshell.file(schema=f.schema)
#from https://academy.ifcopenshell.org/posts/ifcopenshell-optimizer-tutorial/

    def generate_instances_and_references():
        """
        Generator which yields an entity id and 
        the set of all of its references contained in its attributes. 
        """
        for inst in f:
            yield inst.id(), set(i.id() for i in f.traverse(inst)[1:] if i.id())

            
    instance_mapping = {}

    def map_value(v):
        """
        Recursive function which replicates an entity instance, with 
        its attributes, mapping references to already registered
        instances. Indeed, because of the toposort we know that 
        forward attribute value instances are mapped before the instances
        that reference them.
        """
        if isinstance(v, (list, tuple)):
            # lists are recursively traversed
            return type(v)(map(map_value, v))
        elif isinstance(v, ifcopenshell.entity_instance):
            if v.id() == 0:
                # express simple types are not part of the toposort and just copied
                return g.create_entity(v.is_a(), v[0])
    
            
            return instance_mapping[v]
        else:
            # a plain python value can just be returned
            return v

    info_to_id = {}
    
    
    for id in toposort(dict(generate_instances_and_references())):
        inst = f[id]
        info = inst.get_info(include_identifier=False, recursive=True, return_type=frozenset)
        if info in info_to_id:
            mapped = instance_mapping[inst] = instance_mapping[f[info_to_id[info]]]

        else:
            info_to_id[info] = id
            instance_mapping[inst] = g.create_entity(
                inst.is_a(),
                *map(map_value, inst)
            )
    return g


In [3]:
ifcPath="../data/Duplex_A.ifc"
ifc = ifcopenshell.open(ifcPath)

In [4]:
ifcshow(ifc)

 ###### 3D rendering pipe initialisation #####
Display3d class initialization starting ...
Aspect_DisplayConnection created.
Graphic_Driver created.
V3d_Viewer created.
AIS_InteractiveContext created.
V3d_View created
Xw_Window created.
Display3d class successfully initialized.
 ########################################


In [15]:
walls = ifc.by_type('IfcWall')
wall = walls[0]
shape = ifcopenshell.geom.create_shape(settings, wall)#.geometry
# Trsf = shape.Location().Transformation()
# print(gpTrsf2T(Trsf))

# wall.get_info()

shape_tuple(data=<ifcopenshell.ifcopenshell_wrapper.serialized_element_double_precision; proxy of <Swig Object of type 'IfcGeom::SerializedElement< double,double > *' at 0x7fdb41c298d0> >, geometry=<class 'TopoDS_Compound'>, styles=((0.5019607843137255, 0.5019607843137255, 0.5019607843137255, 1.0),))

In [7]:
from ifcopenshell.util.selector import Selector
selector = Selector()

print(ifc.schema)
print(ifc.by_guid('1xS3BCk291UvhgP2dvNMKI')) #found Level 1 in .ifc 


elements = selector.parse(ifc, '@ #1xS3BCk291UvhgP2dvNMKI')

#https://github.com/IfcOpenShell/IfcOpenShell/issues/564#issuecomment-477049228

ifcNew = ifcopenshell.file(schema=ifc.schema)
for element in elements:
    ifcNew.add(element)
ifcNew.write('ifcNew_Duplex_A.ifc')

IFC2X3
#39=IfcBuildingStorey('1xS3BCk291UvhgP2dvNMKI',#33,'Level 1',$,$,#38,$,$,.ELEMENT.,0.)


In [11]:
 OptimizeIFC(ifcNew)

TypeError: 'module' object is not callable

In [4]:
occ_display, start_display, add_menu, add_function_to_menu = init_display()
cam = occ_display.View.Camera()
# display IFC
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
products = ifcNew.by_type("IfcProduct")
for product in products:
    if product.is_a("IfcOpeningElement"): continue
    if product.Representation:
        shape = ifcopenshell.geom.create_shape(settings, product)
        r, g, b, a = shape.styles[0] # the shape color
        color = Quantity_Color(0.6, abs(g), abs(b), Quantity_TOC_RGB)
        display_shape = occ_display.DisplayShape(shape.geometry,color=color, transparency=abs(1 -a),update=True)
start_display() #must be used or kernel crashes

 ###### 3D rendering pipe initialisation #####
Display3d class initialization starting ...
Aspect_DisplayConnection created.
Graphic_Driver created.
V3d_Viewer created.
AIS_InteractiveContext created.
V3d_View created
Xw_Window created.
Display3d class successfully initialized.
 ########################################
